# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

## <span style='color:#2656a3'> ⚙️ Feature View Retrieval

## <span style='color:#2656a3'> 🗄 Model Registry

## <span style='color:#2656a3'> 📮 Retrieving model from Model Registry

## <span style='color:#2656a3'> ✨ Load Batch Data

## <span style='color:#2656a3'> 🤖 Making the predictions